In [ ]:
import math
import logging
import pandas as pd
import altair as alt
import numpy as np
from scipy import ndimage
from astropy.convolution import Gaussian2DKernel

import sys,os,os.path
sys.path.append(os.path.expanduser('~/src/spinorama/src'))

from spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from spinorama.graph import contour_params_default


# df = parse_graphs_speaker('Klipsch', 'Klipsch R41M', 'klippel')
# df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)
logging.basicConfig(level='ERROR')

In [ ]:
def contour_angle_sort(df : pd.DataFrame) -> pd.DataFrame:
    """ reorder from -90 to +270 and not -180 to 180 to be closer to other plots                                                                        
    """
    def a2v(angle):
        if angle == 'Freq':
            return -1000
        elif angle == 'On Axis':
            return 0
        iangle = int(angle[:-1])
        if iangle <-90:
            return iangle+360
        return iangle
    return df.reindex(columns=sorted(df.columns, key=lambda a: -a2v(a)))    


def contour_normalize(df : pd.DataFrame) -> pd.DataFrame:
    # normalize dB values wrt on axis                                                                                                                 
    avg = df['On Axis'].loc[(df.Freq>500)&(df.Freq<12000)].mean()
    for c in df.columns:
        if c != 'Freq':
            df[c] -= avg
    return df


def compute_contour(df):
    dfu = df.copy()
    dfu = contour_angle_sort(dfu)
    dfu = contour_normalize(dfu)
    vrange = [a for a in dfu.columns if a is not 'Freq']
    # print(vrange)
    # melt                                                                                                                                            
    dfu = graph_melt(dfu)
    # 
    # avg = dfu.dB.max()
    # compute numbers of measurements                                                                                                                 
    nm = dfu.Measurements.nunique()
    nf = int(len(dfu.index) / nm)
    logging.debug('unique={:d} nf={:d}'.format(nm,nf))
    # index grid on a log scale log 2 ±= 0.3                                                                                                          
    hrange = np.floor(np.logspace(1.0+math.log10(2), 4.0+math.log10(2), nf))
    # 3d mesh                                                                                                                                         
    af, am = np.meshgrid(hrange, vrange)
    # since it is melted generate slices                                                                                                              
    az = np.array([dfu.dB[nf * i:nf * (i + 1)] for i in range(0, nm)])
    print(np.min(az), np.max(az))
    return (af, am, az)


In [ ]:
def reshape(x, y, z, nscale):
    nx, ny = x.shape
    # expand x-axis and y-axis                                                                                                                        
    lxi = [np.linspace(x[0][i], x[0][i+1], nscale, endpoint=False) for i in range(0, len(x[0])-1)]
    lx = [i for j in lxi for i in j] + [x[0][len(x[0])-1] for i in range(0, nscale)]
    nly = (nx-1)*nscale+1
    print(y)
    ly = np.linspace(np.min(y.ravel()), np.max(y.ravel()), nly)
    # on this axis, cheat by 1% to generate round values that are better in legend                                                                    
    # round off values close to those in ykeep                                                                                                        
    ykeep = [20, 30, 100, 200, 300, 400, 500,
             1000, 2000, 3000, 4000, 5000,
             10000, 20000]
    def close(x1, x2, ykeep):
        for z in ykeep:
            if abs((x1-z)/z) < 0.01 and z<x2:
                ykeep.remove(z)
                return z
        return x1
    lx2 = [close(lx[i], lx[i+1], ykeep) for i in range(0,len(lx)-1)]
    lx2 = np.append(lx2, lx[-1])
    # build the mesh                                                                                                                                  
    rx, ry = np.meshgrid(lx2, ly)
    # copy paste the values of z into rz                                                                                                              
    rzi = np.repeat(z[:-1], nscale, axis=0)
    rzi_x, rzi_y = rzi.shape
    rzi2 = np.append(rzi, z[-1]).reshape(rzi_x+1, rzi_y)
    rz = np.repeat(rzi2, nscale, axis=1)
    # print(rx.shape, ry.shape, rz.shape)                                                                                                             
    return (rx, ry, rz)


def compute_contour_smoothed(dfu, nscale=5):
    # compute contour                                                                                                                                 
    x, y, z = compute_contour(dfu)
    if len(x) == 0 or len(y) == 0 or len(z) == 0:
        return (None, None, None)
    # std_dev = 1                                                                                                                                     
    kernel = Gaussian2DKernel(1, mode='oversample', factor=10)
    # extend array by x5                                                                                                                              
    rx, ry, rz = reshape(x, y, z, nscale)
    # convolve with kernel                                                                                                                            
    rzs = ndimage.convolve(rz, kernel.array, mode='mirror')
    # return                                                                                                                                          
    return (rx, ry, rzs)

In [ ]:
def angle_range(angles):
    a = [int(a[:-1]) for a in angles if a != 'Freq' and a != 'On Axis']
    return np.max(a)-np.min(a)
    

# explicitly set ticks for X and Y axis                                                                                                               
xTicks = [i*10 for i in range(2,10)] + [i*100 for i in range(1,10)] + [i*1000 for i in range(1, 21)]
yTicks = [-180+10*i for i in range(0,37)]

def graph_contour_common(af, am, az, graph_params):
    try:
        width = graph_params['width']
        height = graph_params['height']
        # more interesting to look at -3/0 range                                                                                                      
        speaker_scale = None
        if 'contour_scale' in graph_params.keys():
            speaker_scale = graph_params['contour_scale']
        else:
            speaker_scale = contour_params_default['contour_scale']
        speaker_range = ['violet', 'blue', 'lightblue', 'green', 'yellow', 'orange', 'red']

        # flatten and build a Frame                                                                                                                   
        freq = af.ravel()
        angle = am.ravel()
        db = az.ravel()
        if (freq.size != angle.size) or (freq.size != db.size):
            logging.debug('Contour: Size freq={:d} angle={:d} db={:d}'.format(freq.size, angle.size, db.size))
            return None

        source = pd.DataFrame({'Freq': freq, 'Angle': angle, 'dB': db})

        # tweak ratios and sizes                                                                                                                      
        chart = alt.Chart(source)
        # classical case is 200 points for freq and 36 or 72 measurements on angles                                                                   
        # check that width is a multiple of len(freq)                                                                                                 
        # same for angles
        r = angle_range(angle)
        height *= r/360

        # build and return graph                                                                                                                      
        # print('w={0} h={1}'.format(width, height))
        if width/source.shape[0] < 2 and height/source.shape[1] < 2:
            chart = chart.mark_point()
        else:
            chart = chart.mark_rect()

        chart = chart.transform_filter(
            'datum.Freq>400'
        ).transform_calculate(
            z='floor(datum.dB/3)*3'
        ).encode(
            alt.X('Freq:O',
                  scale=alt.Scale(type="log", base=10, nice=True),
                  axis=alt.Axis(
                      format='.0s',
                      labelAngle=0,
                      values=xTicks,
                      title='Freq (Hz)',
                      labelExpr="datum.value % 100 ? null : datum.label")),
            alt.Y('Angle:O',
                  axis=alt.Axis(
                      format='.0d',
                      title='Angle',
                      values=yTicks,
                    labelExpr="datum.value % 30 ? null : datum.label"),
                  sort=None),
            alt.Color('z:Q',
                      scale=alt.Scale(#scheme='plasma',
                                      range=speaker_range,
                                      domain=speaker_scale,
                                      nice=True))
        ).properties(
            width=width,
            height=height
        )
        return chart
    except KeyError as ke:
        logging.warning('Failed with {0}'.format(ke))
        return None


def graph_contour(df, graph_params):
    af, am, az = compute_contour(df)
    if af is None or am is None or az is None:
        logging.error('contour is None')
        return None
    return graph_contour_common(af, am, az, graph_params)


def graph_contour_smoothed(df, graph_params):
    af, am, az = compute_contour_smoothed(df)
    if af is None or am is None or az is None:
        logging.warning('contour is None')
        return None
    if np.max(np.abs(az)) == 0.0:
        logging.warning('contour is flat')
        return None
    return graph_contour_common(af, am, az, graph_params)

In [ ]:
params = contour_params_default
params['width'] = 400
params['height'] = 400
params['contour_scale'] = [-21, -18, -15, -12, -9, -6, -3, 0]

def compare(df1, title1, df2, title2):
    V1 = df1['SPL Vertical_unmelted']
    V2 = df2['SPL Vertical_unmelted']
    GV1 = graph_contour(V1, params).properties(title='{0} Vertical'.format(title1))
    GV2 = graph_contour(V2, params).properties(title='{0} Vertical'.format(title2))
    H1 = df1['SPL Horizontal_unmelted']
    H2 = df2['SPL Horizontal_unmelted']
    GH1 = graph_contour(H1, params).properties(title='{0} Horizontal'.format(title1))
    GH2 = graph_contour(H2, params).properties(title='{0} Horizontal'.format(title2))
    return (GV1 | GV2) & (GH1 | GH2)

In [ ]:
df1 = parse_graphs_speaker('KEF', 'KEF LS50', 'klippel')
df2 = parse_graphs_speaker('KEF', 'KEF LS50', 'princeton')

compare(df1, 'LS50 ASR', df2, 'LS50 3D3A')


In [ ]:
df1 = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')
df2 = parse_graphs_speaker('Genelec', 'Genelec 8351A', 'princeton')

compare(df1, '8341A ASR', df2, '8351A 3D3A')

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

af, am, az = compute_contour(df1['SPL Vertical_unmelted'])

im = plt.imshow(az, interpolation='bilinear', cmap=cm.RdYlGn,
                origin='lower', extent=[-10, 10, -10, 10],
                vmax=abs(az).max(), vmin=-abs(az).max())

plt.show()


In [ ]:
graph_contour_smoothed(df1['SPL Vertical_unmelted'], contour_params_default)